# Imports

In [ ]:
import numpy as np
import pandas as pd
import pyodbc
from math import sqrt
import seaborn as sns
import matplotlib.pyplot as plt
import math
from itertools import combinations
from collections import defaultdict
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
%matplotlib inline

# Data

In [ ]:
data = pd.read_csv('export-sales-report-20250202.csv')
data.head()

## proccesing the datetime

In [ ]:
data["PersianDate"] = data["PersianDate"].str.split("/").str[1] + data["PersianDate"].str.split("/").str[2]
data.head()

In [ ]:
lst = data['PersianDate'].to_list()

In [ ]:
grouped = data.groupby(["PersianDate", "BuyerId"])["ProductId"].apply(list).reset_index()
grouped.head()

# FP - growth

## grouping the data

In [ ]:
productid = data['ProductId'].unique().tolist()

data["PersianDate"] = data["PersianDate"].str.split("/").str[1] + data["PersianDate"].str.split("/").str[2]

grouped = data.groupby(["PersianDate", "BuyerId"])["ProductId"].apply(list).reset_index()

ap_df = grouped[grouped['BuyerId'] == 50]

transactions = ap_df['ProductId'].tolist()

## the algortihm

In [ ]:
all_products = sorted(set(item for sublist in transactions for item in sublist))  # لیست تمام محصولات
onehot_encoded_data = []
for transaction in transactions:
    onehot_encoded_data.append([1 if product in transaction else 0 for product in all_products])

onehot_df = pd.DataFrame(onehot_encoded_data, columns=all_products)

frequent_itemsets = fpgrowth(onehot_df, min_support=0.05, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

frequent_itemsets
rules

# Recommender Function

In [ ]:
def fpgrowth_recommender(rules: pd.DataFrame, item_list: list):
    filtered_rules = pd.DataFrame()

    for item in item_list:
        filtered_rules = pd.concat([filtered_rules, rules[rules['antecedents'].apply(lambda x: item in x)]])

    filtered_rules = filtered_rules[filtered_rules['confidence'] > 0.1]

    recommended_items = (filtered_rules[['consequents', 'confidence']]
                         .explode('consequents')
                         .sort_values(by='confidence', ascending=False))

    return [item for item in recommended_items['consequents'].tolist() if item not in item_list]



def drop (lsrecom):
    rlst = []
    for i in lsrecom:
        if i not in rlst:
            rlst.append(i)
    return rlst




x = int(input("enter the first product id: "))
p_list = []
p_list.append(x)
products_df = pd.read_excel('p_id_name.xlsx')
while True:
    recommended = fpgrowth_recommender(rules, p_list)
    recommendeddf = pd.DataFrame(drop(recommended), columns=['ProductId'])
    recommendeddf = recommendeddf.merge(products_df, left_on='ProductId', right_on='ID', how='left')
    print(recommendeddf[['ProductId', 'Name']].head(10))  
    new_item = input("Enter the next product id or 'exit' to finish: ")
    if new_item.lower() == 'exit':
        break
    try:
        new_item = int(new_item)
        if new_item not in p_list:
            p_list.append(new_item)
        else:
            print("code already entered")
    except ValueError:
        print("Invalid input. Please enter a valid product id")

In [ ]:
def drop (lsrecom):
    rlst = []
    for i in lsrecom:
        if i not in rlst:
            rlst.append(i)
    return rlst

In [ ]:
x = int(input("enter the first product id: "))
p_list = []
p_list.append(x)
products_df = pd.read_excel('p_id_name.xlsx')
while True:
    recommended = fpgrowth_recommender(rules, p_list)
    recommendeddf = pd.DataFrame(drop(recommended), columns=['ProductId'])
    recommendeddf = recommendeddf.merge(products_df, left_on='ProductId', right_on='ID', how='left')
    print(recommendeddf[['ProductId', 'Name']].head(10))  
    new_item = input("Enter the next product id or 'exit' to finish: ")
    if new_item.lower() == 'exit':
        break
    try:
        new_item = int(new_item)
        if new_item not in p_list:
            p_list.append(new_item)
        else:
            print("code already entered")
    except ValueError:
        print("Invalid input. Please enter a valid product id")